In [1]:
!pip install konlpy
import re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
     |████████████████████████████████| 3.8MB 57.0MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
reviews = pd.read_csv('drive/My Drive/datasets/review_coupang_1.csv')
reviews = reviews.loc[:,['content']]

In [3]:
reviews.sample(frac=1).reset_index(drop=True)

,content
0,양이 적다 맛은 괜찮은데.. 양이 너무 작아요. 2개 들어 있는데 다 먹어도 부족할...
1,신선하고 맛있어요 샐러리 통채로 사려하니 손질하기 귀찮기도 하고 양도 많을 것 같아...
2,간편식으로ㅠ부담없이 좋아요\n맛있어요..신선하고~~ 맛 만족도 예상보다 맛있어요
3,맛있어요 늘 맛이게 잘 먹고 있는 떡볶이입니다! 냉동실에 쟁여두고 뭐 먹을까 고민될...
4,고춧가루도 국산이고 새콤하고 와사비의 알싸한 맛까지 아주 좋아요.\n단짠 유행이지만...
...,...
146887,애기들이좋아할것같음 작아서 한입에먹기 편리하고 애기들이 좋아할것같아요
146888,너무 기대를 했는지 조금은 밍밍했어요 물보다 우유를 넣고 끓이면 좀 낫고 매운맛은 ...
146889,자취생 아이 어르신계신 집에서는 쟁여놓고 먹기 좋아요 간편하게 과일먹고싶을때 딱좋아
146890,오늘처럼 펑펑 눈오는 날..바로 시식해야겠어요 막내가 좋아하는 라면...저렴하게 잘...


In [4]:
!pip install chatspace

     |████████████████████████████████| 10.7MB 5.9MB/s 


In [7]:
# 띄어쓰기 보정 오픈소스
from chatspace import ChatSpace

spacer = ChatSpace()

Loading JIT Compiled ChatSpace Model


In [21]:
reviews = reviews.dropna()

In [ ]:
reviews

In [23]:
test_data = []
for text in reviews['content']:
  test_data.append(spacer.space(text).replace('\n',''))

In [24]:
test_data = pd.DataFrame(test_data, columns=['review'])
test_data

,review
0,똥손에게 gom gom 밀키트는 구세주!! 한식은 정말 좋아하는데 요리는 못 하거든...
1,우삼겹이 보들보들하고 쫄깃쫄깃해요~ 우삼겹 순두부 ㅋㅋ 처음으로 밀키트를 구입해봤어...
2,추운 겨울 얼큰하게 끓여 소주 한잔^^ 비오는 날 흐린 날 순두부 찌개에 소주 한잔...
3,비 오는 날 순두부 찌개 어떠신가요? ㅎㅎ 또 주문해서 먹었습니다 ^^ 제가 찌개를...
4,반조리 식품은 아니지만 밀키트로 된 상품을 두 번째로 구매했는데 정말 바쁠 때는 안...
...,...
144861,맛 만족도 괜찮아요
144862,맛 만족도 괜찮아요
144863,맛 만족도 괜찮아요
144864,맛 만족도 괜찮아요


In [25]:
keyword_list = {'맛': ['맛','불맛', '감칠맛', '쓴맛', '향', '단맛', '신맛', '냄새', '끝맛', '짠맛', '꼬순내', '햄맛',
                      '햄향', '닭맛', '불향', '풋내', '조미료', '끝향', '쏘스맛', '닭내', '파맛', '존맛', '핵존맛', '텁텁한맛'],
                '가격':['가격', '금액', '값', '단가', '할인', '가성', '득템', '창렬', '혜자', '정가', '사기'],
                '양':['용량', '양은','양이', '양도', '양으로', '개수', '인분', '분량', '물량', '사이즈', '수량','남았', '남아'],
                '품질':['품질', '상태', '아이스', '얼음', '비닐', '아이스팩', '뾱뾱', '뽁뽁','드라이아이스', '아이스박스', '보냉', '상자',
                      '종이팩', '종이', '상온', '실온','상한','재료'
                        '소재', '박스', '비닐봉투', '밀봉', '보냉팩', '배송', '발송', '택배', '배달', '검사', '퀄리티', '퀄러티','환경',
                       '검증', '신뢰', '신선',  '유통', '유효', '기한', '싱싱', '상한', '부패', '냄새'],
                '간편':['간편', '간단', '초보', '방법', '쉬움', '어려움', '단시간', '출퇴근시간', '간편식', '전문식','편리', '손쉽', '혼밥', '꽝손', 
                      '불편', '똥손', '금손', '편의', '손쉬'],
                '외관':['외관', '육안', '디자인', '실물', '겉보기', '보고', '보임', '볼때', '보고', '맛보다',
                       '푸디', '색감', '색깔', '모양새','이쁘', '예쁘','이쁜', '예쁜', '이뻐요', '예뻐요'],
                '웰빙':['웰빙', 'msg', '인스턴트', '나트륨', '특수', '식품', '즉석식품', '건강', '다이어트', '다이어터', '헬스', '첨가', '식단', '단백질',
                      '영양', '효과', '예방', '체중', '영양소', '탄수화물', '칼로리', '성분', '소화효소', '체중감량', '면역', '근육', '야맹증',
                      '질환', '보조', '미용', '운동', '에너지', '호르몬', '칼슘', '성인병', '갱년기', '성장', '비료', '노화', 
                      '치매', '효능', '염분', '두뇌', '불균형', '뇌', '알츠하이머', '코로나', '영양분', '소화','비만','뚱뚱', '살빼', '녹색','동물']}

In [26]:
for keyword in keyword_list:
  test_data[keyword] = -1
test_data

,review,맛,가격,양,품질,간편,외관,웰빙
0,똥손에게 gom gom 밀키트는 구세주!! 한식은 정말 좋아하는데 요리는 못 하거든...,-1,-1,-1,-1,-1,-1,-1
1,우삼겹이 보들보들하고 쫄깃쫄깃해요~ 우삼겹 순두부 ㅋㅋ 처음으로 밀키트를 구입해봤어...,-1,-1,-1,-1,-1,-1,-1
2,추운 겨울 얼큰하게 끓여 소주 한잔^^ 비오는 날 흐린 날 순두부 찌개에 소주 한잔...,-1,-1,-1,-1,-1,-1,-1
3,비 오는 날 순두부 찌개 어떠신가요? ㅎㅎ 또 주문해서 먹었습니다 ^^ 제가 찌개를...,-1,-1,-1,-1,-1,-1,-1
4,반조리 식품은 아니지만 밀키트로 된 상품을 두 번째로 구매했는데 정말 바쁠 때는 안...,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
144861,맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
144862,맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
144863,맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1
144864,맛 만족도 괜찮아요,-1,-1,-1,-1,-1,-1,-1


In [41]:
# tokenizing
okt = Okt()
def tokenize(doc):
    doc = spacer.space(doc)
    # 한글 자음, 모음 제거
    doc = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=doc)
    # 특수기호 제거
    doc = re.sub(pattern='[^\w\s]', repl='', string=doc)
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    doc = okt.pos(doc, norm=True, stem=True)
    # 조사 제외
    token = []
    for i in doc:
        if i[1] != 'Josa':
          token.append(i)

    return ['/'.join(t) for t in token]

In [42]:
def vectorize(sentence, encoder):
  l = []
  token = tokenize(sentence)
  l.append(token)
  result = encoder.texts_to_sequences(l)
  return pad_sequences(result, 300)

In [61]:
# 인코더 기존 학습된 모델 사용
import pickle
with open('drive/My Drive/datasets/encoder_v3.pickle', 'rb') as handle:
    encoder = pickle.load(handle)
# saving
# with open('encoder_v3.pickle', 'wb') as handle:
#     pickle.dump(encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
def inject_keyword(text, window):
  # 결과값
  result = {'맛':'', '가격': '', '양':'', '품질':'', '간편':'', '외관':'', '웰빙':''}
  sentences = re.split('[,.;?!]+',text) # 문장 분리 by ",.;?"

  # 맛 ~ 웰빙까지
  for key in keyword_list.keys():
    keys = keyword_list[key] # 키워드 + 연관 키워드 리스트
    injected = '' # 추출할 문장

    for k in keys: 
      for sentence in sentences:
        words = sentence.split(' ') # 각 문장 띄어쓰기 분리
        for word in words:
          if k in word:
            index = words.index(word)
            # 키워드에 해당하는 단어의 인덱스를 기준으로 앞 뒤 몇 글자씩 잘라내서 추출
            # window 파라미터로 몇 글자를 잘라낼 것인지 설정
            # 문맥 상 키워드의 이전 단어들 보다 이후에 등장하는 단어들을 더 많이 추출
            for i in range(1,2):
              if index - i >= 0:
                injected += words.pop(index - i)
                injected += ' '
            # for i in range(window+2):
            #   if index + i < len(words):
            #     injected += words.pop(index + i)
            #     injected += ' '
            index = words.index(word)
            for i in range(window):
              if index < len(words):
                injected += words.pop(index) + ' '
            injected += '.'
                 
      result[key] = injected
      
  return result

In [31]:
print(test_data['review'][0])
print(inject_keyword(test_data['review'][0], 5))

똥손에게 gom gom 밀키트는 구세주!! 한식은 정말 좋아하는데 요리는 못 하거든요. 매번 15, 000원씩 배달 주문하는 것도 무리라는 생각이 들던 차. gom gom 우삼겹 순두부 찌개 밀키트를 발견했습니다. 와 저는 맛있게 먹었어요. 생애 첫 제 손으로 만든 찌개를 먹고 감격. 다른 gom gom 밀키트도 해먹어 볼 예정입니다. 1. 패키지 구성 및 내용 물 양 - 조리 방법은 패키지 겉 종이에 간략하게 적혀있어요. 시키는 대로 만들면 성공입니다. - 패키지에 찌개 소스 + 순두부 + 채소 + 우삼겹만 깔끔히 들어있습니다. >> 채소, 고기 양이 부족한 느낌은 아닙니다. - 추가 재료 없이 1팩 구성으로만 끓여도 양이 푸짐합니다. >> 성인 혼자 먹을 때는 2 ~ 2. 5끼 정도 먹을 양이 만들어지더라고요. >> 식성 좋은 3인이 한 끼 배부르게 먹었습니다. 2. 요리 전 상태 - 찌개 소스는 향과 색이 자극적이라 ‘아! 이건 맛없을 수가 없다.’ 싶더군요. - 순두부는 흔히 마트에 파는 그 순두부입니다. 유통기한 넉넉했어요. - 채소 믹스에는 ‘애호박 + 팽이버섯 + 고추 + 파’가 있고, 물기가 많았지만 신선했습니다. - 우삼겹은 밀키트 구성품으로 있는 것치고 상태가 좋은 편입니다. >> 고기 빛깔이 전체적으로 좋았고, '말캉 + 탱탱 + 촉촉한 상태’ 였습니다. >> 약간의 비릿한 피 + 생고기 냄새만 납니다. 누린내는 거의 안 났어요. >> 비계, 살코기 비율이 괜찮더군요. 3. 재료 추가 없이 밀키트 구성으로만 요리 시 - 패키지에 적힌 조리 방법 그대로 물, 소스, 재료 넣고 센불에 끓이다가 마지막에 순두부만 넣었습니다. - 가장 큰 걱정이 물 조절이었는데, 저는 일반 머그 컵 2. 5컵 분량을 넣었습니다. >> 순두부와 채소에서 수분이 나오기 때문에 굳이 더 물을 넣을 필요는 없어 보이더라고요. - 찌개 소스는 1팩 다 넣었습니다. 자극적일 것 같지만 시키는 데로 다 넣었어요. - 순두부는 덩어리로 넣어도 되지만, 저는 자잘자 잘 하게 으깨

In [63]:
# 테스트용 3000개 데이터 사용
samples = test_data[:3000]
# 사전 학습된 LSTM 모델 사용
model = load_model('drive/My Drive/datasets/LSTM_binary_v2.h5')
for sample in samples['review']:
  inject_result = inject_keyword(sample, 3)
  for key in keyword_list.keys():
    inject_sentence = inject_result[key]
    # 해당 키워드가 없으면 점수는 -1
    if inject_sentence == '':
      samples.loc[sample, key] = -1
    else:
      samples.loc[sample, key] = model.predict(vectorize(inject_sentence, encoder))[0][0] * 100
samples

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,review,맛,가격,양,품질,간편,외관,웰빙
0,똥손에게 gom gom 밀키트는 구세주!! 한식은 정말 좋아하는데 요리는 못 하거든...,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
1,우삼겹이 보들보들하고 쫄깃쫄깃해요~ 우삼겹 순두부 ㅋㅋ 처음으로 밀키트를 구입해봤어...,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
2,추운 겨울 얼큰하게 끓여 소주 한잔^^ 비오는 날 흐린 날 순두부 찌개에 소주 한잔...,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
3,비 오는 날 순두부 찌개 어떠신가요? ㅎㅎ 또 주문해서 먹었습니다 ^^ 제가 찌개를...,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
4,반조리 식품은 아니지만 밀키트로 된 상품을 두 번째로 구매했는데 정말 바쁠 때는 안...,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...
"한끼 먹기 좋아요 2인이 먹기에 적당합니다. 4인 가족이 먹기에 부족하여 고기, 숙주 추가해 먹었습니다. 직접 만들어 먹으면 버리는 야채가 더 많은데, 한끼 먹을 양 만큼 와서 좋습니다. 내용물 중 미니 새송이는 품질이 떨어져서 별 하나 뺍니다. 맛이 있는 편이고, 간 편하여 재구매의사 있습니다.",NaN,99.243307,-1.000000,-1.000000,87.755030,-1.0,-1.0,-1.0
밀푀유나 베는 다 맛있어용 로켓 프레쉬에서 판매하는 밀푀유나베 브랜드 별로 다 먹어본 밀푀유나베 러버입니당 그 중에 곰곰 밀푀유나 베가 가장 포장이 깨끗하고 재료 손질이 잘 되어 있는 것 같아요! 깻잎이나 배추 등 흐물거리거나 상한 거 없이 아주 깔끔하게 왔습니당! 하지만 재료가 약간 부족했어용 이제껏 먹어본 브랜드들보다는 부족한 느낌 육수 만드는 물 양도 좀 적은 느낌이더라구 영 그래도 맛은 최고였어요 참 깨소스? 그거보다 간장 맛 나는 소스가 더 많았으면 좋겠어요,NaN,99.799901,-1.000000,96.112120,58.140868,-1.0,-1.0,-1.0
진짜 편하고 내용물도 신선해요~ 진짜 괜찮아서 재구매 했어요 이제 장보지 않고 밀키트 주문해서 먹으니 쓰레기 안나오고 양도 적절해서 과식 않하게 되고 조리하기 너무 편하고 시간 절약되고 아무튼 신세계에요~ 가성비 좋아서 또 구매할 생각입니다,NaN,-1.000000,99.966657,97.424567,99.168551,-1.0,-1.0,-1.0
와 저는 이거 왜 이제 발견한 거죠...? 진짜 가족들이랑 알차게 먹었어요 ㅎㅎ 다들 국물도 맛있고 구성도 좋다고 해서 진짜 기분좋게 한끼 해결했어요 ! 숙주를 밑에 깔았어야 했는데 까먹고 위에 얹었는데도 상관없이 맛있었어용 재구매 의사 무조건 있구 다들 주저마세요 ㅜㅜ 아 근데 배추가 다른 재료에 비해 적은 느낌이었어요 근데 맛있어서 노상관 짱짱 맛임 더 맛 만족도 예상보다 맛있어요,NaN,99.750727,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0


In [66]:
test = samples[samples['맛'] != -1]
test[test['맛'] < 50]

,review,맛,가격,양,품질,간편,외관,웰빙
노 맛 맛없더 !!!! !!!!!!!! !!! 배민 아이디가 맛 없을시에 침 뱉음 인데 진짜 조오오오오오 오옹나 맛없어요. 아니 그냥 맛이 아예 무 . 무 맛 . 없을 무 . 기름 맛 ? 아니 나 타이머 켜고 조리법 그대로.. 심지어 탐사수 사용해서 물 500미리 물 양도 그대로였는데.. 하. 뭐가 문제였지. 정말 맙소사. 띠로리. 진짜. 정말. 님들 무슨 맛으로 맛있다는 거에요 ? 저 진짜 싱겁게 먹는데 이게 무슨 맛인데요 도대체 ? 순두부 찌개 맛이 원래 이럼 ? 진짜 최소한 스팸이라도 썰어 넣을껄 ㅎ 후회중 ; ㅎ 심지어 순두부 꺼내다가 풍 ! 뒁 ! 떨궈서 온 집안 벽에 고추기름 퐈 ! 튀 ! 했습니다 ㅎ ㅎ ㅎ 엄마한테 등짝 스매싱 터질까봐 벽지 뜯었어요 •_ < 뿌,NaN,4.149332,-1.0,98.215818,-1.000000,-1.000000,-1.000000,-1.0
로켓 프레쉬 배송받고 김치냉장고에 이틀 보관했다가 끓여먹었어요 유통기한 내에 먹었는데 내 입맛이 이상한 건가 약간 시큼한듯한 맛도 나고 상품평보고 기대했는데 생각보다 좀 별로였어요 결국 한끼 먹고 다 버렸네요 입맛의 개인차이가 있겠지만 저한테는 맛이 그닥 그랬습니다.,NaN,22.696026,-1.0,-1.000000,24.246752,-1.000000,11.646160,-1.0
비주얼도 좋지만 막상 먹어보면 국물이 괜찮아요. 보통 마트에서 파는 순두부 양념은 조미료 맛이 엄청나게 나잖아요. 그래서 좀 그런데.. 이건 그게 별로 안 느껴져요. 여기 쿠킹 박스 제대로 하네~~ 하는 생각이 들 정도에요~ ^^ ㅋㅋㅋ 완전... 한번 드셔보세요~ ^^ 강추입니다~ 쏘주 한잔 생각나는 국물맛이었어요~ 맛 만족도 예상보다 맛있어요,NaN,29.874101,-1.0,-1.000000,94.395828,-1.000000,-1.000000,-1.0
양도 풍ㅈ부합니다 단지 짠듯하고 조미료 맛이 납니다 맛 만족도 괜찮아요,NaN,49.107575,-1.0,99.791270,-1.000000,-1.000000,-1.000000,-1.0
내가 물양을 조금 많이 잡았나 ㅎ 시키는데루 하긴 했는데.. 맛이 없진 않은데 또 파는 맛 같지도 않았어요. 지는 요리도 못하는 주제에 식당에서 파는 음식은 또 기준이 높아서 제입에는 그렇게 느껴졌어요.,NaN,26.336145,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
...,...,...,...,...,...,...,...,...
후추향강한 양념 채소중당근상해서 옴 당근이 쉬어서 옴 상했음 반품하기 애매해서 그냥 당근 빼고 만들었는데 색감이 양념은 후추맛이 강한 듯,NaN,13.914376,-1.0,-1.000000,-1.000000,-1.000000,96.402323,-1.0
느타리버섯이 상했는지 냄새가 역해서 버섯은 다 버렸어요 ㅜㅜ 맛은 쏘쏘 맛 만족도 괜찮아요,NaN,33.122149,-1.0,-1.000000,9.094360,-1.000000,-1.000000,-1.0
고기에서 잡내나고 버섯에서도 쉰내났어요 앞으론 안시킬래요 맛 만족도 예상보다 맛 없어요,NaN,41.533911,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
채소는 신선했습니다. 간편하게 조리해먹긴 괜찮아요. 근데 소스에 후추가 너무 많이 들어가 있고 돼지고기 양념같은 조미료 향이 너무 강하네요.,NaN,10.495286,-1.0,-1.000000,98.267531,99.767679,-1.000000,-1.0


In [70]:
test = samples[samples['가격']!= -1]
test[test['가격'] > 50]

,review,맛,가격,양,품질,간편,외관,웰빙
먹느라 사진을 못찍었어요..ㅠ 맛있게 잘 먹었습니다 가격도 저렴하고 끓이기 편하고 최고에요! 전 호박을 더 썰어넣고 먹었어요 재료 담겨 있던 플라스틱 통에 물양이 적혀 있어서 참 좋으네요~ 세심한 배려같아서 좋았습니다 개인적으로 곰곰상품은 좋은 것 같아요~ 맛 만족도 예상보다 맛있어요,NaN,98.089749,98.466253,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
"와 혼밥 할 때 끝판왕 정말 맛있습니다. 혼밥을 주로 하는 사람에게는 최고의 메뉴가 아닌가 싶네요! 우선 양이 꽤 됩니다. 일반 시중에 파는 상품들보다 양이 많은 것 같아요. 서너번은 먹을 수 있는 양입니다. 맛나네요. 재료가 좋습니다. 사진에도 나오지만 호박, 버섯, 파! 그리고 가장 중요한 우삼겹!!!! 맛나 맛나 흐엉 ㅠㅠ 그리고 매콤합니다. 많이 맵지 않지만 맛있게 매콤한 그런 맛!!! 가격도 착하고 맛도 착하고, 잘 먹었습니다!!! 쿠팡 체험단 이벤트로 상품을 무료로 제공 받아 작성한 구매 후기입니다.",NaN,99.951351,97.090584,-1.000000,-1.000000,99.721694,-1.000000,-1.0
냉동 아니라 더 좋아요 오늘 4인가족 2팩해서 먹으려다 남편이랑 아들만 밥 먹게 되어서 1팩만 끊였어요. 맛은 두 남자 좋다고 해요 ^^ 양도 많아요 저희 식구가 먹성이 좋아 두 팩을 시켰는데 양 좋아요.. 가격 좋구요. 담에도 또 만날 의양 있어요.,NaN,92.344624,99.157554,99.517411,-1.000000,-1.000000,-1.000000,-1.0
"직장 맘으로 진짜 요리 못하는데, 우와 요즘은 너무 세상이 좋아졌다는 걸 느꼈어요.. 순두부 찌게 사려면 하나 하나 재료들 사고 돈은 돈대로 들고 맛도 보장 못하던 저였는데 우연히 곰공 우삼겹 순두부 찌게 키드 보고 구매했는데 대박~~ 완젼~~~ 맛있어요. 우리집 식구들이 사기 아니냐고~~~ 이렇게 손쉽게 잘 만들면 누가 요리를 하겠냐고 ^^ 양도 푸짐하고 무엇보다 야채가 신선합니다. 하나 하나 개별 포장된 야채들이 너무 신선해서 바로 해먹지 않고 며칠 두었는데도 그대로에요. 신선한 재료를 넣으니 신선도가 유지 되는 듯해요. 양도 푸짐하고 2인분보다 더 더 양이 많아요. 저 미리 또 시켜먹으려고 담아 놨습니다. 이렇게 맛있고 푸짐한 제품 만들어 주셔서 감사해요. 맛 만족도 예상보다 맛있어요",NaN,99.339092,82.018477,99.882776,96.924543,99.273294,99.715167,-1.0
한끼 가격에 두끼 맛있게 먹었어요! 우선 가격도 저렴하고 재료도 간단하게 들어있어서 조리하기 너무 쉬웠어요! 배송받고 이틀 뒤에 먹었는데도 상한 것 하나 없이 모든 재료 신선했구 2명이서 먹기 충분할만큼 양도 많아요 우삼겹 질기지 않고 부드러웠고 양념도 맛있어서 식당에서 사먹는 맛 나요! 한끼 가격에 두끼를 먹다니 개이득,NaN,99.921715,99.818039,99.579877,24.428079,98.960179,-1.000000,-1.0
...,...,...,...,...,...,...,...,...
간단히 혼자 먹기에 가성비 좋은 구성 간단히 먹기에 최고에요 야채랑 다 들어있으시깐 좋아요 이 가격 구성이면 가성비 좋댜고 생각해요 둘이 먹기에 작아요 ~ 저는 둘이서 안주로 먹엇어요 고기랑 야채 건져먹고 들어있는 국수랑 수제비 먹고 햇반 돌려서 국물을 좀 퍼내고 밥 김가루 참기름 계란 넣고 죽까지 해먹었어요~ 안주로 말고 한끼로 둘이. 먹기에 부족해욤 두 팩은 먹어야 되실듯요~ 넉넉히 드실거라면 두 팩하세요 소스 얘기가 잇던데 참 소스는 괜찮고 피넛 소스는 연하고 좀 느끼한 맛이에요 월남쌈 피넛 소스 마트에 파는 게 훨 맛나요 거기에 칠리소소랑 곁들여 먹엇네요 도움되셧음 좋겠어여,NaN,99.283361,99.755472,-1.000000,-1.000000,99.984956,-1.000000,-1.0
(*´ー｀*) 다른 후기와 같이 썩은 깻잎이 있네요 야채 포장된 것을 봤을 때 깻잎 상태가 괜찮은 줄 알았으나 중간에 썩은 것이 포함 되어 있어요 (*´-`) 고기는 진공포장이 되어 있어서 그런지 한장한 징 띄어내기 불편한 부분이 있어요 ╰ (*´︶`*) ╯그래도 하나 하나 사는 것보단 가성비는 정말 좋습니다 ψ(｀∇´) ψ근데 백탕소스가 빠져있어서 육수에다가 제가 간을 좀 맞췄어요 ㅠ 맛 만족도 괜찮아요,NaN,97.576922,99.843603,-1.000000,86.697561,90.083009,-1.000000,-1.0
"요새 코로 나로 재택근무를 하다보니 집에서 끼니를 먹는 때가 많아졌는데, 요리를 잘 못하는 저에게는 이렇게 재료랑 소스가 다 들어있는 밀키트가 엄청 유용하더라구요 ㅎㅎㅎ 밀푀유나 베는 이자카야 에서만 봤지 실제로 만들어 본 것도 처음인데 설명 따라 하니까 진짜 쉬운 요리에요 제가 냄비 크기를 잘못 골라서 너무 커서 샤브샤브처럼 되어버렸지만...... 좀 작은 냄비에 오밀조밀 담아야 예쁠 거 같아요 ㅠㅠ 야채랑 고기가 비닐에 각각 담겨있고 소스도 양 넉넉하게 들어있어요! 2인분이라 소스 두개 들어있어서 각각 앞접시 할 수 있는 것도 넘 좋았구 무려 칼국수까지 들어있어요 혜자 포장이에요 ㅋㅋㅋㅋㅋㅋ 2인분이라고 되어있는데 밥 한공기 같이 먹으니 셋이 먹어도 넉넉했습니다. 국물도 안짜고 딱 맛있어요!! 재구매의사 있습니다 :) 쿠팡 체험단 이벤트로 상품을 무료로 제공 받아 작성한 구매 후기입니다. 맛 만족도 예상보다 맛있어요",NaN,97.780335,98.222810,95.960450,98.916233,-1.000000,-1.000000,-1.0
대만족! 배송도 역시는 역시! 저녁에 시켰더니 12시 조금 넘어서 도착해서 엄청 놀랐어요 7시쯤 오겠거니 했는데.. 편하기도 하고 직접 재료 다 사서 했을 때남는 재료 등등 여러 가지~ 둘이 먹었는데 남네요~ 구성도 좋고 가격이면 가격 맛이면 맛~~~ 자주 시켜 먹어야겠어요 저같은 요똥에게 진짜 진짜 고마운 쿠팡 프레시~! 오늘도 고맙습니다 ^^ 맛 만족도 예상보다 맛있어요,NaN,91.084129,96.937948,-1.000000,99.101233,-1.000000,-1.000000,-1.0


비고 
 * 아직까지 정확도에 한계가 있다. 키워드 추출 로직도 보완 필요.
 * 키워드 연관 단어 사전 역시 보완 필요
 * POC 나 프로토타입 때 보여주기 용으로는 나쁘지 않은듯.
 * 계산 시간 비용이 많이 든다. 3000개 학습 시 10분 정도, 9만 개 학습 시 5시간

In [ ]:
# 키워드 형태소 분석

for key in keyword_list.keys():
  for word in keyword_list[key]:
    print(okt.pos(word))

In [ ]:
okt.pos(test_data['review'][0])

In [47]:
model_test = load_model('drive/My Drive/datasets/LSTM_binary_v2.h5')

In [65]:
model_test.predict(vectorize('맛없어요', encoder))

array([[0.13574399]], dtype=float32)